# Imports

In [133]:
import os
from pathlib import Path
import pandas as pd



# Parameters and constants

In [134]:
camels_spat_attrs_dir = Path('../../../../../gladwell/hydrology/SUMMA/summa-ml-models/CAMELS_spat_NH')

camels_spat_attrs_file = camels_spat_attrs_dir / ('camels_spat_attributes.csv')
camels_spat_attrs_file_to_nh = camels_spat_attrs_dir / ('camels_spat_attributes_to_nh.csv')

attributes_dict_found = [
    'prec_mean',
    'pet_mean',
    'aridity1_mean',
    'seasonality1_mean',
    'fracsnow1_mean',
    'high_prec_freq',
    'high_prec_dur_mean',
    'low_prec_freq ',
    'low_prec_dur_mean',
    'elev_mean',
    'slope_mean',
    'basin_area',
    'soil_thickness_mean',
    'porosity_mean', 
    'log_permeability_mean',
]

attributes_dict_computed = [
    'lai_mean_month_max',
    'lai_mean_month_diff',
    'sand_layer',
    'silt_layer',
    'clay_layer',

    # forest_frac          --> lc2_mixed_forest_fractio (???). No -> Select either lc1_ or lc2, 
    # and find the ratio of forest pixels to all pixels per basin

    # soil_conductivity    --> Maybe use log_permeability_mean as a proxy or convert to 
    # conductivity somehow (https://en.wikipedia.org/wiki/Permeability_(materials_science)#Relation_to_hydraulic_conductivity)
]

# Functions

In [135]:
# Function to rename columns based on the provided rules
def rename_columns(col):
    if col.startswith('CAN'):
        # Remove prefix and pad the remaining part to 7 places with trailing zeros
        new_col = col[4:].ljust(7, '0')
        return new_col
    elif col.startswith('USA'):
        # Remove prefix and pad the remaining part to 8 places with trailing zeros
        new_col = col[4:].ljust(8, '0')
        return new_col
    else:
        return col
    
def compute_lai_mean_month_max_diff(attrs_df, attrs_df_nh):
    # Filter the rows related to lai_mean_month
    lai_mean_month_rows = attrs_df[attrs_df['Attribute'].str.contains('lai_mean_month_')].copy()

    # Convert the values to numeric
    lai_mean_month_rows.iloc[:, 1:] = lai_mean_month_rows.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
    
    # Compute the maximum and minimum values among the months for each column
    lai_mean_month_max = lai_mean_month_rows.iloc[:, 1:].max()
    lai_mean_month_min = lai_mean_month_rows.iloc[:, 1:].min()
    
    # Compute the difference between the max and min values
    lai_mean_month_diff = lai_mean_month_max - lai_mean_month_min
    
    # Create new rows for lai_mean_month_max and lai_mean_month_diff
    lai_mean_month_max_row = pd.DataFrame([['lai_mean_month_max'] + lai_mean_month_max.tolist()],
                                          columns=attrs_df.columns)
    lai_mean_month_diff_row = pd.DataFrame([['lai_mean_month_diff'] + lai_mean_month_diff.tolist()],
                                           columns=attrs_df.columns)
    
    # Append the new rows to attrs_df_nh
    attrs_df_nh = pd.concat([attrs_df_nh, lai_mean_month_max_row, lai_mean_month_diff_row], ignore_index=True)
    
    return attrs_df_nh
    
def compute_fraction_mean(attrs_df, attrs_df_nh, fraction_type):
    # Identify rows that match the pattern for the specified fraction type
    fraction_rows = attrs_df[attrs_df['Attribute'].str.contains(fr'{fraction_type}_\d+-\d+cm_mean')].copy()
    
    # Ensure we have identified rows
    if fraction_rows.empty:
        print(f"No matching {fraction_type} fraction rows found.")
        return attrs_df_nh
    
    # Convert the values to numeric, handling non-numeric values gracefully
    for col in fraction_rows.columns[1:]:
        fraction_rows[col] = pd.to_numeric(fraction_rows[col], errors='coerce')
   
    # Convert the values from fractional representation to percentages (e.g., 800 to 80%)
    fraction_rows.iloc[:, 1:] = fraction_rows.iloc[:, 1:] / 10.0
    
    # Drop rows where all values are NaN
    fraction_rows.dropna(how='all', subset=fraction_rows.columns[1:], inplace=True)
    
    # Ensure there are valid rows to process
    if fraction_rows.empty:
        print(f"No valid {fraction_type} fraction values after conversion.")
        return attrs_df_nh
    
    # Compute the mean value for each column
    fraction_means = fraction_rows.iloc[:, 1:].mean()

    # Check if all values in fraction_means are NaN
    if fraction_means.isna().all():
        print(f"Computed {fraction_type}_mean is NaN.")
        return attrs_df_nh

    # Create a new row with the computed mean values for each column
    new_row = pd.DataFrame([[f'{fraction_type}_layer_mean'] + fraction_means.tolist()],
                           columns=['Attribute'] + fraction_means.index.tolist())
    
    # Append the new row to attrs_df_nh
    attrs_df_nh = pd.concat([attrs_df_nh, new_row], ignore_index=True)
    
    return attrs_df_nh

# Load attributes file

In [136]:
# Read the CSV file, specifying dtype as string to avoid mixed types warning
attrs_df = pd.read_csv(camels_spat_attrs_file, dtype=str)

# Only keep the columns 'Attribute' and the ones with a pattern ABC_code like CAN_01AD002 or USA_14306500
attrs_df = attrs_df.filter(regex='Attribute|^[A-Z]{3}_[0-9A-Z]+$').copy()

# Apply the rename function to the columns
attrs_df.rename(columns=rename_columns, inplace=True)

# Filter by column 'Attribute' values to be in attributes_dict_found
attrs_df_nh = attrs_df[attrs_df['Attribute'].isin(attributes_dict_found)].copy()

# Compute and add the rows for the computed attributes
# 'lai_mean_month_max' and 'lai_mean_month_diff' are computed from the 'lai_mean_month_XX' rows
attrs_df_nh = compute_lai_mean_month_max_diff(attrs_df, attrs_df_nh)

# 'sand_frac', 'silt_frac', and 'clay_frac' are computed from the 'TYPE_XX-YYcm_mean' rows
attrs_df_nh = compute_fraction_mean(attrs_df, attrs_df_nh, 'sand')
attrs_df_nh = compute_fraction_mean(attrs_df, attrs_df_nh, 'silt')
attrs_df_nh = compute_fraction_mean(attrs_df, attrs_df_nh, 'clay')

## Format and save new attribute file

In [137]:
# Transpose the DataFrame
attrs_df_nhT = attrs_df_nh.transpose().copy()

# Set the first row as the header
attrs_df_nhT.columns = attrs_df_nhT.iloc[0]
attrs_df_nhT = attrs_df_nhT[1:]

# Reset the index to make 'Attribute' a column again
attrs_df_nhT.reset_index(inplace=True)

# Rename 'index' to 'basin'
attrs_df_nhT.rename(columns={attrs_df_nhT.columns[0]: 'basin'}, inplace=True)

# Save the final DataFrame to a CSV file
attrs_df_nhT.to_csv(camels_spat_attrs_file_to_nh, index=False)

print(attrs_df_nhT.columns)

# Display the final DataFrame
display(attrs_df_nhT)

Index(['basin', 'aridity1_mean', 'fracsnow1_mean', 'seasonality1_mean',
       'low_prec_dur_mean', 'high_prec_freq', 'high_prec_dur_mean',
       'prec_mean', 'pet_mean', 'basin_area', 'elev_mean', 'slope_mean',
       'soil_thickness_mean', 'porosity_mean', 'log_permeability_mean',
       'lai_mean_month_max', 'lai_mean_month_diff', 'sand_layer_mean',
       'silt_layer_mean', 'clay_layer_mean'],
      dtype='object', name='Attribute')


Attribute,basin,aridity1_mean,fracsnow1_mean,seasonality1_mean,low_prec_dur_mean,high_prec_freq,high_prec_dur_mean,prec_mean,pet_mean,basin_area,elev_mean,slope_mean,soil_thickness_mean,porosity_mean,log_permeability_mean,lai_mean_month_max,lai_mean_month_diff,sand_layer_mean,silt_layer_mean,clay_layer_mean
0,01AD002,0.18798583750094697,0.2756233080498433,0.15468573009104025,2.693663292088489,15.857452966714906,1.0893032384690873,1080.056879631484,233.75525736030443,14691.61894,362.03506767343833,4.029331134624624,0.7809965276359849,0.1331932769477868,-14.37731088910784,4.476095,4.058043,39.032694,27.675981,14.716443
1,01AD003,0.17771905684226516,0.2894199021283202,0.16851650097590484,2.7707018195321202,16.366039409246905,1.1108986615678775,1095.7515151515152,227.5035173160173,1343.736223,353.94557528636614,5.309244119201183,0.7881686573002401,0.1607692298264457,-14.657692212324877,4.683582,4.393957,32.331455,28.878228,17.072509
2,01AE001,0.31516898447471303,0.2555185982980571,0.1476173286789044,2.8450272479564034,17.25731887110362,1.1016393442622952,1010.4347371174471,239.5193238294533,2249.524543,276.4003361537538,4.708683230092109,0.6957798995914186,0.1580769221226755,-14.707692219660832,4.237068,3.900777,29.837864,31.546744,15.110833
3,01AF007,0.421913933668009,0.24524128909027543,0.14515365834232732,2.812354988399072,16.891807035295518,1.0959097320169253,1070.9710144927535,233.81368885869566,323.8344727,267.085233314709,3.614874795562039,0.7370588078218347,0.1574999978765842,-14.349999904632568,4.474721,4.123323,26.809677,31.412054,20.29823
4,01AF009,0.12263759217704563,0.2690464221134831,0.13456896062270401,2.793576017130621,16.580279077982865,1.1053763440860216,1102.198113207547,227.16194968553458,183.6754339,337.75361356232406,6.686667928116752,0.6857232627628734,0.189999997615814,-15.199999809265137,4.886292,4.595422,27.616713,29.495778,19.319038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1629,14309500,0.004136225478121908,0.0581173905850098,-0.9188400456171943,5.575639204545454,19.89531914024191,1.4031578947368422,1640.6544943820224,363.3109857794944,225.9326068,739.8640906806634,20.123505046005175,0.6259550458929511,0.1538461529864714,-14.28461529658391,4.182288,2.671913,25.312107,28.414839,23.543059
1630,14316700,0.23315401880713255,0.15202128002119514,-0.7678638062615898,4.798497267759563,16.39427943250643,1.2942583732057416,1592.4171122994653,349.9737967914439,587.6714412,942.100099887361,20.79754919219184,0.5990408126679645,0.1246000017598272,-13.14799997329712,4.653633,3.278226,34.925531,27.094363,15.696824
1631,14325000,-0.005607113418416518,0.027193922852899996,-0.9384284575663833,5.612433862433862,20.068681318681318,1.3807728557964185,1871.7749287749289,351.73357371794873,442.658586,653.6522656628739,17.227649122500043,0.6989173997840991,0.1734999986365437,-14.439999866485596,4.508547,2.798837,28.68656,28.100129,22.856641
1632,14362250,1.3813068482275603,0.04256914814163236,-0.8143255035207517,5.62764705882353,20.256054756054755,1.301482701812191,966.2258064516129,395.36321824596774,41.64237572,973.2661870503597,22.882219908714475,0.5164515587591356,0.030000000726432,-13.700000286102297,4.011342,2.380259,27.849074,29.936394,20.444932
